In [104]:
from openai import OpenAI
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # 
import pandas as pd
import json
import copy
import tqdm
import numpy as np

In [3]:
path_train = "../data/SHROOM_unlabeled-training-data-v2/train.model-aware.v2.json"

In [4]:
df = pd.read_json(path_train)
df

,hyp,tgt,src,ref,task,model
0,Of or pertaining to the language of a particul...,"Of or pertaining to everyday language , as opp...","There are blacktips , silvertips , bronze whal...",tgt,DM,ltg/flan-t5-definition-en-base
1,Not coercive ; not involving coercion,Not coercive ; free of coercion,Mr. obama signed executive orders requiring al...,tgt,DM,ltg/flan-t5-definition-en-base
2,To express or express by words ; to express by...,To depict or portray .,Disloyal ? / the word is too good to paint out...,tgt,DM,ltg/flan-t5-definition-en-base
3,Having the power to authoritatively speak or w...,Having a commanding style .,"He instructed us in that booming , authoritati...",tgt,DM,ltg/flan-t5-definition-en-base
4,Without a scot .,"Without consequences or penalties , to go free...",To get off scot-free . What is the meaning of ...,tgt,DM,ltg/flan-t5-definition-en-base
...,...,...,...,...,...,...
29995,Many hotels offer a room rent and a room rent ...,Ironing damp clothes can help them dry. Many h...,Loko mpahla yi tsakama u nga yi ayina ivi yi o...,either,MT,facebook/nllb-200-distilled-600M
29996,"This year, plans for delivery of vaccines to m...",Plans for vaccines to be delivered to the hist...,Дар соли ҷорӣ нақшаҳои таҳвили ваксинаҳо ба ми...,either,MT,facebook/nllb-200-distilled-600M
29997,Arukan still wants to be the hero of the exped...,They are still trying to determine just how la...,ⴰⵔⵓⴽⴰⵏ ⵙⵓⵍ ⵜⵍⵉⵏ ⴰⴷⵃⴷⵉⴷⴻⵏ ⵍⵃⴰⵊⴻⵎ ⵏ ⵓⵎⵏⴳⴳⴰⵙ ⴷ ⵎⴰ...,either,MT,facebook/nllb-200-distilled-600M
29998,Ronald Reagan moved the photographer to UCLA M...,The photographer was transported to Ronald Rea...,రోనాల్డ్ రీగన్ యుసిఎల్‌ఎ మెడికల్ సెంటర్‌కు ఫోట...,either,MT,facebook/nllb-200-distilled-600M


In [159]:
client = OpenAI(api_key="sk-XXX") #enter your key

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return client.chat.completions.create(**kwargs)

def create_sample(row):
    if row["task"] == "PG":
        context = row['src']
    else: #i.e. task == "MT" or task == "DM":
        context = row['tgt']
    
    return {"context":context,"sentence":row['hyp']}

# def answer_to_label(answer:str):
#     formatted = answer.strip().lower()
#     assert formatted in ["yes","no"], f"Got wrong answer: '{answer}'"
#     return "Hallucination" if formatted == "no" else "Not Hallucination"

def answer_to_label(answer:float):    
    return "Hallucination" if answer < 0.5 else "Not Hallucination"

def detect(samples):
    batch_instruction = {
        "role": "system",
        "content":
        'You will be provided with a Sentence \
        and your task is to rate the consistency of that sentence to \
        that of the provided Context. Your answer must be only \
        a number between 0.0 and 1.0 rounded to the nearest two \
        decimal places where 0.0 represents no consistency and \
        1.0 represents perfect consistency and similarity. \
        Reply with a valid JSON in following format: {"answers":{"<pair_id>": <float>} }. Example: {"answers":{"0":0.7,"12":0.33} }. Array of answers should contain reply for each Context/Sentence pair.'
    }
    for i, s in enumerate(samples):
        s["pair_id"] = str(i)
    completion = completion_with_backoff(
        model="gpt-3.5-turbo-1106",
        messages=[
            batch_instruction,
            {"role": "user", "content": "Pairs: " + json.dumps(samples,indent=1)}
        ],
        temperature=0,
        response_format = {"type": "json_object"},
        seed=42
    )
    answers = json.loads(completion.choices[0].message.content)["answers"].values()
    return [(answer_to_label(a),a) for a in answers], completion

In [160]:
batch_size = 32 # turned out to be the best size, larger ones result in worse detections
batched_df = [df[i:i+batch_size] for i in range(0,len(df),batch_size)]

In [161]:
df_labels = []
for batch in tqdm.tqdm(batched_df,total=len(batched_df)):
    samples = batch.apply(create_sample,axis=1).to_list()
    labels,completion = detect(samples)
    assert len(labels) == len(samples), f"Returned result contains {len(labels)}, not {len(samples)}"
    df_labels.append(labels) # store result per batch

  1%|▏         | 13/938 [01:27<1:28:27,  5.74s/it]